In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import cv2
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('Model/model_hand.h5')

# map the integer values with the characters.
word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}


def predict_alphabet(img):
    img_copy = img.copy()
    img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2RGB)
    img = cv2.resize(np.float32(img), (400,440))
    img_copy = cv2.GaussianBlur(np.float32(img_copy), (7,7), 0)
    img_gray = cv2.cvtColor(np.float32(img_copy), cv2.COLOR_BGR2GRAY)
    _, img_thresh = cv2.threshold(np.float32(img_gray), 100, 255, cv2.THRESH_BINARY_INV)

    img_final = cv2.resize(np.float32(img_thresh), (28,28))
    img_final =np.reshape(np.float32(img_final), (1,28,28,1))
    img_pred = word_dict[np.argmax(model.predict(img_final))]
    return img_pred

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit = predict_alphabet(im)
        self.label.configure(text= str(digit))

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()